# 3.1: 自动微分量子门和线路

由自动微分量子门构成的量子线路，被称变分量子线路 （variational quantum circuit，简称VQC，或称自动微分量子线路（ADQC）、参数化量子线路（PQC））

更一般的概念：变分量子算法（VQA）

## 量子态制备

以量子态制备（quantum state preparation）任务为例：将给定初态利用量子线路演化至目标量子态

量子态制备需极大化目标态与末态间的保真度（fidelity）：

$f = |\langle\varphi'|\varphi^{\text{target}}\rangle|$

* 由量子态归一性可得，保真度满足：$0 \leq f \leq 1$
* $f = 1$时，两个量子态间仅相差一个相位因子，可认为二者表示同一个态
* $f = 0$时，两个量子态正交

这里的变分量子门将主要采用任意单体么正门的旋转表示（参考2.2节）:

$\hat{U} = e^{i\eta}\hat{R}^{(z)}(\beta)\hat{R}^{(y)}(\gamma)\hat{R}^{(z)}(\delta) = \begin{pmatrix} e^{i(\eta-\beta/2-\delta/2)}\cos\frac{\gamma}{2} & -e^{i(\eta-\beta/2+\delta/2)}\sin\frac{\gamma}{2} \\ e^{i(\eta+\beta/2-\delta/2)}\sin\frac{\gamma}{2} & e^{i(\eta+\beta/2+\delta/2)}\cos\frac{\gamma}{2} \end{pmatrix}$

> 这里的变分参数有 $\eta, \beta, \gamma, \delta$四个，分别对应于旋转门的相位、$R^{(z)}$门的角度、$R^{(y)}$门的角度和$R^{(z)}$门的角度

> 取决于优化方法和损失函数，$e^{i\eta}$ 可以省略

## Code

In [ ]:
import torch

In [ ]:
#|export tensor_gates.functional
from typing import LiteralString
from tensor_network.utils import map_float_to_complex

def pauli_operator(*, pauli: LiteralString['X', 'Y', 'Z', "ID"], 
                   double_precision: bool = False,
                   force_complex: bool = False) -> torch.Tensor:
    assert pauli in ['X', 'Y', 'Z', "ID"], f"Invalid Pauli operator: {pauli}"
    if double_precision:
        dtype_complex = torch.complex128
        if force_complex:
            dtype_default = dtype_complex
        else:
            dtype_default = torch.float64
    else:
        dtype_complex = torch.complex64
        if force_complex:
            dtype_default = dtype_complex
        else:
            dtype_default = torch.float32

    if pauli == 'X':
        return torch.tensor([[0, 1], [1, 0]], dtype=dtype_default)
    elif pauli == 'Y':
        return torch.tensor([[0, -1j], [1j, 0]], dtype=dtype_complex)
    elif pauli == 'Z':
        return torch.tensor([[1, 0], [0, -1]], dtype=dtype_default)
    elif pauli == 'ID':
        return torch.eye(2, dtype=dtype_default)
    else:
        raise Exception("Unreachable code")


def rotate(params: torch.Tensor) -> torch.Tensor:
    assert isinstance(params, torch.Tensor), "params must be a torch.Tensor"
    assert params.shape == (4,), "params must be a 4-element vector"
    dtype = params.dtype
    assert dtype in [torch.float32, torch.float64], "params must be float32 or float64"
    gate_dtype = map_float_to_complex(dtype=dtype)
    alpha, beta, delta, theta = params[0], params[1], params[2], params[3]
    gate = torch.ones((2, 2), device=params.device, dtype=gate_dtype)
    gate[0, 0] = torch.exp(1j * (delta - alpha / 2 - beta / 2)) * torch.cos(theta / 2)
    gate[0, 1] = -torch.exp(1j * (delta - alpha / 2 + beta / 2)) * torch.sin(theta / 2)
    gate[1, 0] = torch.exp(1j * (delta + alpha / 2 - beta / 2)) * torch.sin(theta / 2)
    gate[1, 1] = torch.exp(1j * (delta + alpha / 2 + beta / 2)) * torch.cos(theta / 2)
    return gate


### VQC Example

![VQC Example](./images/vqc_example.png)